### Dependencies

In [2]:
# !pip install pypdf langchain langchain_core langchain_community langchain_huggingface langchain_groq sentence_transformers transformers langchain_google_genai faiss-cpu langchain_openai grandalf pymupdf

### Loading PDF

In [5]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("GSTsmartGuide.pdf")

In [6]:
docs = loader.load()
docs[0]

Document(metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2024-02-27T11:53:51+05:30', 'source': 'GSTsmartGuide.pdf', 'file_path': 'GSTsmartGuide.pdf', 'total_pages': 1321, 'format': 'PDF 1.6', 'title': 'CHAPTER 1', 'author': 'Abha', 'subject': '', 'keywords': '', 'moddate': '2024-02-27T13:03:27+05:30', 'trapped': '', 'modDate': "D:20240227130327+05'30'", 'creationDate': "D:20240227115351+05'30'", 'page': 0}, page_content='')

In [7]:
len(docs)

1321

In [9]:
docs[10]

Document(metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2024-02-27T11:53:51+05:30', 'source': 'GSTsmartGuide.pdf', 'file_path': 'GSTsmartGuide.pdf', 'total_pages': 1321, 'format': 'PDF 1.6', 'title': 'CHAPTER 1', 'author': 'Abha', 'subject': '', 'keywords': '', 'moddate': '2024-02-27T13:03:27+05:30', 'trapped': '', 'modDate': "D:20240227130327+05'30'", 'creationDate': "D:20240227115351+05'30'", 'page': 10}, page_content='10 \nContents \n \nChapter \nTopic Subject wise \nSection of \nCGST \nAct \nRule of  \nCGST \nRules \nPage \nNo. \nChapter 21 \nAdvance Ruling \n96 to 101 \n103 to 107 \n543 \nChapter 22 \nAppeals and Revision \n107 to 115 108 to 115 \n562 \nChapter 23 \nOffence and Penalties \n122 to 138 \n162 \n593 \nChapter 24 \nAnti-Profiteering Measures \n171 \n122 to 137 \n620 \nChapter 25 \nJob Work under GST \n143 \n \n632 \nChapter 26 \nElectronic Way Bill \n \n138 to 138D 651 \nChapter 27 \nAuthorised Representatives \n116 \n

### Contextual Retrieval (a way of chunk enrichment).

In [10]:
def create_enriched_chunks(docs, chunk_size=1000, overlap=200):
    """Create contextually enriched chunks from documents loaded with LangChain."""
    all_chunks = []

    for doc in docs:
        text = getattr(doc, 'page_content', str(doc)).strip()
        if not text:
            continue

        source = doc.metadata.get('source', "unknown") if hasattr(doc, 'metadata') else "unknown"
        page_number = doc.metadata.get('page', None) if hasattr(doc, 'metadata') else None

        chunks = []
        for i in range(0, len(text), chunk_size - overlap):
            chunk_text = text[i:i + chunk_size]
            if chunk_text:
                chunks.append({
                    "text": chunk_text,
                    "source": source,
                    "start_pos": i,
                    "page": page_number,
                })

        for i, chunk in enumerate(chunks):
            prev_chunk = chunks[i-1]["text"] if i > 0 else ""
            next_chunk = chunks[i+1]["text"] if i < len(chunks) - 1 else ""

            enriched_chunk = {
                "text": chunk["text"],
                "source": chunk["source"],
                "position": chunk["start_pos"],
                "page": chunk.get("page"),
                "context": {
                    "previous": prev_chunk[-200:] if prev_chunk else "",
                    "next": next_chunk[:200] if next_chunk else ""
                }
            }

            all_chunks.append(enriched_chunk)

    return all_chunks


In [11]:
enriched_chunks = create_enriched_chunks(docs)

In [12]:
enriched_chunks

[{'text': "Bharat's \nG S T   \nSmart Guide  \n \n \n \n \n \n \n \n \nC\nO\nM\nM\nI\nT\nT\nE\nD\nT\nO\nQ\nU\nA\nL\nI\nT\nY\nA\nN\nD\nE\nX\nC\nE\nL\nL\nE\nN\nC\nE",
  'source': 'GSTsmartGuide.pdf',
  'position': 0,
  'page': 1,
  'context': {'previous': '', 'next': ''}},
 {'text': 'Opportunity rarely knocks twice. For years, \nyou have dreamt to be an AUTHOR. Here is \njust the right time for you to fulfil your dreams \ninto reality. Bharat opens its doors to Authors \nfor a wide range of books. \nBharat are one of the leading law book \npublishers and need no introduction. Bharat is a \nfamiliar word with the chartered accountants, \ncompany secretaries, corporate executives, \nconsultants, lawyers, students, etc. \nBharat invites proposals from authors having \nready manuscripts or those interested in \nwriting books to please get in touch with us.',
  'source': 'GSTsmartGuide.pdf',
  'position': 0,
  'page': 2,
  'context': {'previous': '', 'next': ''}},
 {'text': "Bharat's \nG S T 

In [17]:
enriched_chunks[101]

{'text': ' the first two \nmonths shall be debited solely for the purposes of offsetting the liability \nfurnished in that quarter’s FORM GSTR-3B. However, any amount left after \nfiling of that quarter’s FORM GSTR-3B may either be claimed as refund or may \nbe used for any other purpose in subsequent quarters. In case of cancellation of \nregistration of such person during any of the first two months of the quarter, he is \nstill required to furnish return in FORM GSTR-3B for the relevant tax period. \n25.6 Applicability of Interest: \nIn case FORM GSTR-3B for the quarter is furnished beyond the due date, \ninterest would be payable as per the provisions of Section 50 of the CGST Act on \nthe tax liability net of ITC. Interest payable, if any, shall be paid through FORM \nGSTR-3B. \n25.7 Applicability of Late Fee: \nLate fee is applicable for delay in furnishing of return/details of outward \nsupply as per the provision of Section 47 of the CGST Act. As per the Scheme, \nthe requireme